# Regresión Logística: Detección de SPAM

En este ejercico se muestran los fundamentos de la regresión logística planteando uno de los primeros problemas que fueron solucionados mediante el uso de técnicas de Machine Learning: La detección de SPAM.

## Enunciado del ejercicio

Se propone la construcción de un sistema de aprendizaje automático capaz de predicir si un correo determinado se corresponde con un correo SPAM o no, para ello se utilizará el siguiente DataSet:

##### [2007/ TREC Public Spam Corpus](https://plg.uwaterloo.ca/~gvcormac/treccorpus07/)

The corpus trec07p contains 75,419 messages:

    25220 ham
    50199 spam

These messages constitute all the messages delivered to a particular
server between these dates:

    Sun, 8 Apr 2007 13:07:21 -0400
    Fri, 6 Jul 2007 07:04:53 -0400

In [6]:
#En esta fase se clasifica el preprocesamiento de correos electronicos que posen codigo HTML

from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.fed = []   # Ingesta de datos
        
    def handle_data(self, d): # Ingerir datos
        self.fed.append(d)
        
    def get_data(self):
        return ''.join(self.fed)


In [16]:
# Esta función se encarga de eliminar los tags en HTML que se encuentren en el texto de los correos electrónicos
def strip_tags(html):
    s = MLStripper()
    s.feed(html)  # Alimentar de los html
    return s.get_data()

In [18]:
# Ejemplo de eliminación de los tags HTML de un texto
t = '<tr><td align="left"><ahref="../../issues/51/16.html#article">Phrack World News</a><td>'
strip_tags(t)

'Phrack World News'

Ademas de eliminar los posibles tags HTML que se encuentran en el correo electrónico, deben realizarse otras acciones para evitar que los mensajes contengan ruido inecesario. Entre ellas se ecnuentra la eliminación de los signos de puntuación, eliminar de los posibles campos de correo electrónico que no sean relevantes o eliminación de los afijos de una palabra manteniendo únicamente la raíz de la misma (Stemming). La clase que se muestra a continuación realiza estas transformaciónes.

In [29]:
import email
import string
import nltk

class Parser:
    def __init__(self):
        self.stemmer = nltk.PorterStemmer()
        self.stopwords = ste(nltk.corpus.stopwords.words('english'))
        self.punctuation = list(string.punctuation)
    
    def parse(self, email_path):
        """Parse and email."""
        with open(email_path, errors = 'ignore') as e:
            msg = email.message_from_file(e)
        return None if not msg else self.get_email_content(msg)

    def get_email_content(self, msg):
        """Extract the email content."""
        subject = self.tokenize(msg['Subject']) if msg['Subject'] else []
        body = self.get_email_body(msg.get_payload(),
                                  msg.get_content_type())
        content_type = msg.get_content_type()
        # Retornar el contenido del email
        return{"subject": subject,
               "body": body,
               "content_type": content_type}

    def get_email_body(self, payload, content_type):
        """Extraxt the body of the email"""
        body = []
        if type(payload) is str and content_type == 'text/plane':
            return self.tokenize(payload)
        elif type(payload) is str and content_type == "text/html":
            return self.tokenize(strip_tags(payload))
        elif type(patload) is list:
            for p in payload:
                body += self.get_email_body(p.get_payload(),
                                           p.get_content_type())
                return body

    def tokenize(self, text):
        """Transform a text string in tokens. Perfomr two main actions, clean the punctuation symbols and do stemming of the text"""
        for c in self.punctuation:
            text = text.replace(c, "")
        text = text.replace("\t", " ")
        text = text.replace("\n", " ")
        tokens = list(filter(None, text.split("")))
        # Stemming of ther tokens
        return [self.stemmer.stem(w) for w in tokens if w not in self.stopword]

Lectura de un correo en formato .raw

In [36]:
inmail = open("datasets/datasets/trec07p/data/inmail.1").read()
print(inmail)

From RickyAmes@aol.com  Sun Apr  8 13:07:32 2007
Return-Path: <RickyAmes@aol.com>
Received: from 129.97.78.23 ([211.202.101.74])
	by speedy.uwaterloo.ca (8.12.8/8.12.5) with SMTP id l38H7G0I003017;
	Sun, 8 Apr 2007 13:07:21 -0400
Received: from 0.144.152.6 by 211.202.101.74; Sun, 08 Apr 2007 19:04:48 +0100
Message-ID: <WYADCKPDFWWTWTXNFVUE@yahoo.com>
From: "Tomas Jacobs" <RickyAmes@aol.com>
Reply-To: "Tomas Jacobs" <RickyAmes@aol.com>
To: the00@speedy.uwaterloo.ca
Subject: Generic Cialis, branded quality@ 
Date: Sun, 08 Apr 2007 21:00:48 +0300
X-Mailer: Microsoft Outlook Express 6.00.2600.0000
MIME-Version: 1.0
Content-Type: multipart/alternative;
	boundary="--8896484051606557286"
X-Priority: 3
X-MSMail-Priority: Normal
Status: RO
Content-Length: 988
Lines: 24

----8896484051606557286
Content-Type: text/html;
Content-Transfer-Encoding: 7Bit

<html>
<body bgcolor="#ffffff">
<div style="border-color: #00FFFF; border-right-width: 0px; border-bottom-width: 0px; margin-bottom: 0px;" align="